In [1]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos en formato de tabla (DataFrame)
import numpy as np  # Para operaciones matemáticas y manejo de arreglos multidimensionales
from sklearn.model_selection import train_test_split  # Para dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # Para normalizar y codificar datos
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a columnas específicas
from sklearn.neural_network import MLPRegressor  # Para crear un modelo de red neuronal para regresión
import matplotlib.pyplot as plt  # Para crear gráficos y visualizaciones 
import plotly.graph_objects as go  # Para crear gráficos interactivos y visualizaciones avanzadas

# Carga del conjunto de datos desde un archivo CSV
dataset = pd.read_csv('orders.csv')  # Carga los datos de pedidos

# Conversión de las fechas a formato de fecha y hora
dataset['shipped_date'] = pd.to_datetime(dataset['shipped_date'])  # Convierte la columna 'shipped_date' a tipo fecha
dataset['required_date'] = pd.to_datetime(dataset['required_date'])  # Convierte la columna 'required_date' a tipo fecha

# Cálculo del tiempo de entrega en horas
dataset['delivery_time'] = (dataset['shipped_date'] - dataset['required_date']).dt.total_seconds() / 3600  # Calcula el tiempo de entrega en horas

# Eliminación de filas con tiempo de entrega nulo
dataset = dataset.dropna(subset=['delivery_time'])  # Elimina filas donde 'delivery_time' es nulo

# Extracción del día de la semana y la hora de envío
dataset['day_of_week'] = dataset['shipped_date'].dt.dayofweek  # Agrega el día de la semana como una nueva columna
dataset['shipped_hour'] = dataset['shipped_date'].dt.hour  # Agrega la hora de envío como una nueva columna

# Separación de características (X) y variable objetivo (y)
X = dataset.drop(columns=["order_id", "delivery_time"])  # Elimina columnas que no se utilizarán y X es la variable donde almacena el entrenamiento.
y = dataset["delivery_time"]  # Define la variable objetivo como 'delivery_time' y Y es la variable donde se almacena el proceso a mostrar.

# Definición de características numéricas y categóricas
numeric_features = ['freight', 'day_of_week', 'shipped_hour']  # Características numéricas a normalizar
categorical_features = ['ship_country', 'ship_address']  # Características categóricas a codificar

# Preprocesamiento de datos: escalado y codificación
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Escala características numéricas
        ('cat', OneHotEncoder(), categorical_features)  # Codifica características categóricas
    ]
)

# Aplicación del preprocesamiento
X_processed = preprocessor.fit_transform(X)  # Aplica las transformaciones definidas en el preprocesador

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)  # Divide los datos en entrenamiento (80%) y prueba (20%)

# Definición del modelo de red neuronal
model = MLPRegressor(
    hidden_layer_sizes=(256, 128),  # Define la arquitectura de la red con dos capas ocultas
    activation='relu',  # Usa la función de activación ReLU
    solver='adam',  # Usa el optimizador Adam para el entrenamiento
    max_iter=500,  # Número máximo de iteraciones para el entrenamiento
    random_state=42  # Semilla para asegurar la reproducibilidad de los resultados
)

# Entrenamiento del modelo
model.fit(X_train, y_train)  # Ajusta el modelo a los datos de entrenamiento

# Evaluación del modelo en el conjunto de prueba
loss = model.score(X_test, y_test)  # Calcula la precisión del modelo en los datos de prueba
print(f"Precisión en el conjunto de prueba (R^2): {loss}")  # Imprime la precisión

# Predicción en el conjunto de prueba
y_pred = model.predict(X_test)  # Realiza predicciones sobre los datos de prueba
print("Predicciones y valores verdaderos:")
for pred, true in zip(y_pred[:10], y_test[:10]):  # Muestra las primeras 10 predicciones
    print(f"Predicción: {pred:.2f} horas, Valor verdadero: {true:.2f} horas")

# Gráfico de comparación entre predicciones y valores verdaderos
fig_comparacion = go.Figure()
fig_comparacion.add_trace(go.Scatter(
    x=list(range(len(y_test))),  # Índices de las predicciones
    y=y_test,  # Valores verdaderos
    mode='markers',
    name='Valor verdadero'  # Etiqueta para la leyenda
))
fig_comparacion.add_trace(go.Scatter(
    x=list(range(len(y_pred))),  # Índices de las predicciones
    y=y_pred,  # Predicciones
    mode='markers',
    name='Predicción'  # Etiqueta para la leyenda
))
fig_comparacion.update_layout(
    title='Comparación de Predicciones y Valores Verdaderos',  # Título del gráfico
    xaxis_title='Índice',  # Título del eje x
    yaxis_title='Tiempo de entrega (horas)'  # Título del eje y
)
# Guardar el gráfico en un archivo HTML
fig_comparacion.write_html('comparacion_predicciones.html')

# Cálculo de residuos
residuals = y_test.values - y_pred  # Calcula la diferencia entre valores verdaderos y predicciones

# Histograma de residuos
fig_residuos = go.Figure(data=[go.Histogram(x=residuals)])  # Crea un histograma de residuos
fig_residuos.update_layout(
    title='Histograma de Residuos (MLPRegressor)',  # Título del histograma
    xaxis_title='Residuos',  # Título del eje x
    yaxis_title='Frecuencia'  # Título del eje y    
)
# Guardar el histograma en un archivo HTML
fig_residuos.write_html('histograma_residuos.html')

# Recuperar los datos de prueba originales
dataset_test = dataset.iloc[y_test.index]  # Obtiene las filas originales de los datos de prueba

# Calcular la hora predicha de envío
dataset_test['predicted_shipped_hour'] = (dataset_test['shipped_date'] + pd.to_timedelta(y_pred.flatten(), unit='h')).dt.hour  # Calcula la hora de envío predicha

# Histograma de la distribución de horas de llegada predicha
fig_distribucion_horas = go.Figure(data=[go.Histogram(x=dataset_test['predicted_shipped_hour'], nbinsx=24)])  # Crea un histograma de horas
fig_distribucion_horas.update_layout(
    title='Distribución de Horas de Llegada Predicha (MLPRegressor)',  # Título del histograma
    xaxis_title='Hora del Día',  # Título del eje x
    yaxis_title='Número de Envíos'  # Título del eje y
)
# Guardar el gráfico en un archivo HTML
fig_distribucion_horas.write_html('distribucion_horas.html')


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Precisión en el conjunto de prueba (R^2): -0.07722329212146573
Predicciones y valores verdaderos:
Predicción: -764.51 horas, Valor verdadero: -600.00 horas
Predicción: -616.17 horas, Valor verdadero: -528.00 horas
Predicción: -876.26 horas, Valor verdadero: -576.00 horas
Predicción: -297.12 horas, Valor verdadero: -456.00 horas
Predicción: -415.84 horas, Valor verdadero: -552.00 horas
Predicción: -430.31 horas, Valor verdadero: -576.00 horas
Predicción: -404.66 horas, Valor verdadero: -528.00 horas
Predicción: -182.59 horas, Valor verdadero: -600.00 horas
Predicción: -615.14 horas, Valor verdadero: -120.00 horas
Predicción: -330.50 horas, Valor verdadero: -480.00 horas


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_32552\3343063150.py:105: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

